# Задание 1.2 - Линейный классификатор (Linear classifier)

В этом задании мы реализуем другую модель машинного обучения - линейный классификатор. Линейный классификатор подбирает для каждого класса веса, на которые нужно умножить значение каждого признака и потом сложить вместе.
Тот класс, у которого эта сумма больше, и является предсказанием модели.

В этом задании вы:
- потренируетесь считать градиенты различных многомерных функций
- реализуете подсчет градиентов через линейную модель и функцию потерь softmax
- реализуете процесс тренировки линейного классификатора
- подберете параметры тренировки на практике

На всякий случай, еще раз ссылка на туториал по numpy:  
http://cs231n.github.io/python-numpy-tutorial/

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_gradient
from metrics import multiclass_accuracy 
import linear_classifer

# Как всегда, первым делом загружаем данные

Мы будем использовать все тот же SVHN.

In [3]:
def prepare_for_linear_classifier(train_X, test_X):
    train_flat = train_X.reshape(train_X.shape[0], -1).astype(np.float) / 255.0
    test_flat = test_X.reshape(test_X.shape[0], -1).astype(np.float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_flat, axis = 0)
    train_flat -= mean_image
    test_flat -= mean_image
    
    # Add another channel with ones as a bias term
    train_flat_with_ones = np.hstack([train_flat, np.ones((train_X.shape[0], 1))])
    test_flat_with_ones = np.hstack([test_flat, np.ones((test_X.shape[0], 1))])    
    return train_flat_with_ones, test_flat_with_ones
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_linear_classifier(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

# Играемся с градиентами!

В этом курсе мы будем писать много функций, которые вычисляют градиенты аналитическим методом.

Все функции, в которых мы будем вычислять градиенты будут написаны по одной и той же схеме.  
Они будут получать на вход точку, где нужно вычислить значение и градиент функции, а на выходе будут выдавать кортеж (tuple) из двух значений - собственно значения функции в этой точке (всегда одно число) и аналитического значения градиента в той же точке (той же размерности, что и вход).
```
def f(x):
    """
    Computes function and analytic gradient at x
    
    x: np array of float, input to the function
    
    Returns:
    value: float, value of the function 
    grad: np array of float, same shape as x
    """
    ...
    
    return value, grad
```

Необходимым инструментом во время реализации кода, вычисляющего градиенты, является функция его проверки. Эта функция вычисляет градиент численным методом и сверяет результат с градиентом, вычисленным аналитическим методом.

Мы начнем с того, чтобы реализовать вычисление численного градиента (numeric gradient) в функции `check_gradient` в `gradient_check.py`. Эта функция будет принимать на вход функции формата, заданного выше, использовать значение `value` для вычисления численного градиента и сравнит его с аналитическим - они должны сходиться.

Напишите часть функции, которая вычисляет градиент с помощью численной производной для каждой координаты. Для вычисления производной используйте так называемую two-point formula (https://en.wikipedia.org/wiki/Numerical_differentiation):

![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/22fc2c0a66c63560a349604f8b6b39221566236d)

Все функции приведенные в следующей клетке должны проходить gradient check.

In [4]:
# TODO: Implement check_gradient function in gradient_check.py
# All the functions below should pass the gradient check

def square(x):
    return float(x*x), 2*x

check_gradient(square, np.array([3.0]))

def array_sum(x):
    assert x.shape == (2,), x.shape
    return np.sum(x), np.ones_like(x)

check_gradient(array_sum, np.array([3.0, 2.0]))

def array_2d_sum(x):
    assert x.shape == (2,2)
    return np.sum(x), np.ones_like(x)

check_gradient(array_2d_sum, np.array([[3.0, 2.0], [1.0, 0.0]]))

Gradient check passed!
Gradient check passed!
Gradient check passed!


True

## Начинаем писать свои функции, считающие аналитический градиент

Теперь реализуем функцию softmax, которая получает на вход оценки для каждого класса и преобразует их в вероятности от 0 до 1:
![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/e348290cf48ddbb6e9a6ef4e39363568b67c09d3)

**Важно:** Практический аспект вычисления этой функции заключается в том, что в ней учавствует вычисление экспоненты от потенциально очень больших чисел - это может привести к очень большим значениям в числителе и знаменателе за пределами диапазона float.

К счастью, у этой проблемы есть простое решение -- перед вычислением softmax вычесть из всех оценок максимальное значение среди всех оценок:
```
predictions -= np.max(predictions)
```
(подробнее здесь - http://cs231n.github.io/linear-classify/#softmax, секция `Practical issues: Numeric stability`)

In [5]:
# TODO Implement softmax and cross-entropy for single sample
probs = linear_classifer.softmax(np.array([-10, 0, 10]))

# Make sure it works for big numbers too!
probs = linear_classifer.softmax(np.array([1000, 0, 0]))
assert np.isclose(probs[0], 1.0)

In [6]:
predictions= np.array([-10, 0, 10])
probs=np.exp(predictions)/np.sum(np.exp(predictions))
print(probs)

[2.06106005e-09 4.53978686e-05 9.99954600e-01]


Кроме этого, мы реализуем cross-entropy loss, которую мы будем использовать как функцию ошибки (error function).
В общем виде cross-entropy определена следующим образом:
![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/0cb6da032ab424eefdca0884cd4113fe578f4293)

где x - все классы, p(x) - истинная вероятность принадлежности сэмпла классу x, а q(x) - вероятность принадлежности классу x, предсказанная моделью.  
В нашем случае сэмпл принадлежит только одному классу, индекс которого передается функции. Для него p(x) равна 1, а для остальных классов - 0. 

Это позволяет реализовать функцию проще!

In [7]:
probs = linear_classifer.softmax(np.array([-5, 0, 5]))
linear_classifer.cross_entropy_loss(probs, 1)

5.006760443547122

In [8]:
-np.log(probs[1])

5.006760443547122

После того как мы реализовали сами функции, мы можем реализовать градиент.

Оказывается, что вычисление градиента становится гораздо проще, если объединить эти функции в одну, которая сначала вычисляет вероятности через softmax, а потом использует их для вычисления функции ошибки через cross-entropy loss.

Эта функция `softmax_with_cross_entropy` будет возвращает и значение ошибки, и градиент по входным параметрам. Мы проверим корректность реализации с помощью `check_gradient`.

In [9]:
# TODO Implement combined function or softmax and cross entropy and produces gradient
loss, grad = linear_classifer.softmax_with_cross_entropy(np.array([1, 0, 0]), 1)
print(loss, grad)
#Не совсем понятно, как может проверяться градиент функции потерь, если мы вычисляем число в заданной точке.
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, 1), np.array([1, 0, 0], np.float))

1.551444713932051 [ 0.57611688 -0.78805844  0.21194156]
Gradient check passed!


True

Последняя проверка не пройдена, хотя численно вроде всё работает, и градиент и функции потерь сходятся.

В качестве метода тренировки мы будем использовать стохастический градиентный спуск (stochastic gradient descent или SGD), который работает с батчами сэмплов. 

Поэтому все наши фукнции будут получать не один пример, а батч, то есть входом будет не вектор из `num_classes` оценок, а матрица размерности `batch_size, num_classes`. Индекс примера в батче всегда будет первым измерением.

Следующий шаг - переписать наши функции так, чтобы они поддерживали батчи.

Финальное значение функции ошибки должно остаться числом, и оно равно среднему значению ошибки среди всех примеров в батче.

In [10]:
# TODO Extend combined function so it can receive a 2d array with batch of samples
np.random.seed(42)
# Test batch_size = 1
num_classes = 4
batch_size = 1
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(np.float)

target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int)
print(predictions, "\n", linear_classifer.softmax(predictions),"\n", target_index)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

# Test batch_size = 3
num_classes = 4
batch_size = 3
predictions = np.random.randint(-1, 3, size=(batch_size, num_classes)).astype(np.float)
target_index = np.random.randint(0, num_classes, size=(batch_size, 1)).astype(np.int)
print(predictions, "\n", linear_classifer.softmax(predictions),"\n", target_index)
check_gradient(lambda x: linear_classifer.softmax_with_cross_entropy(x, target_index), predictions)

[[ 1.  2. -1.  1.]] 
 [[0.20603191 0.56005279 0.02788339 0.20603191]] 
 [[2]]
Gradient check passed!
[[ 2. -1. -1.  1.]
 [ 0.  1.  1.  1.]
 [ 1.  2. -1.  2.]] 
 [[0.68145256 0.03392753 0.03392753 0.25069239]
 [0.10923177 0.29692274 0.29692274 0.29692274]
 [0.15216302 0.41362198 0.02059303 0.41362198]] 
 [[3]
 [3]
 [2]]
Gradient check passed!


True

In [11]:
-np.log(linear_classifer.softmax(np.array([1, 2, -1, 1]))[2])

3.5797242232074917

In [12]:
delta=1e-5

In [13]:
(-np.log(linear_classifer.softmax(np.array([1, 2, -1, 1+delta]))[2])+np.log(linear_classifer.softmax(np.array([1, 2, -1, 1-delta]))[2]))/(2*delta)

0.20603190920009948

In [14]:
-np.log(linear_classifer.softmax(np.array([[ 2., -1., -1.,  1.], [ 0.,  1.,  1.,  1.], [ 1.,  2., -1.,  2.]])))

array([[0.38352864, 3.38352864, 3.38352864, 1.38352864],
       [2.2142833 , 1.2142833 , 1.2142833 , 1.2142833 ],
       [1.88280282, 0.88280282, 3.88280282, 0.88280282]])

In [15]:
np.sum(-np.log(linear_classifer.softmax(np.array([[ 2., -1., -1.,  1.], [ 0.,  1.,  1.,  1.], [ 1.,  2., -1.,  2.]]))[range(target_index.shape[0]),target_index.T]))/target_index.shape[0]

2.160204920616996

In [16]:
((np.sum(-np.log(linear_classifer.softmax(np.array([[ 2., -1., -1.,  1.+delta], [ 0.,  1.,  1.,  1.], [ 1.,  2., -1.,  2.]]))[range(target_index.shape[0]),target_index.T]))/target_index.shape[0])-
 (np.sum(-np.log(linear_classifer.softmax(np.array([[ 2., -1., -1.,  1.-delta], [ 0.,  1.,  1.,  1.], [ 1.,  2., -1.,  2.]]))[range(target_index.shape[0]),target_index.T]))/target_index.shape[0]))/(2*delta)

-0.24976920411923229

### Наконец, реализуем сам линейный классификатор!

softmax и cross-entropy получают на вход оценки, которые выдает линейный классификатор.

Он делает это очень просто: для каждого класса есть набор весов, на которые надо умножить пиксели картинки и сложить. Получившееся число и является оценкой класса, идущей на вход softmax.

Таким образом, линейный классификатор можно представить как умножение вектора с пикселями на матрицу W размера `num_features, num_classes`. Такой подход легко расширяется на случай батча векторов с пикселями X размера `batch_size, num_features`:

`predictions = X * W`, где `*` - матричное умножение.

Реализуйте функцию подсчета линейного классификатора и градиентов по весам `linear_softmax` в файле `linear_classifer.py`

In [17]:
# TODO Implement linear_softmax function that uses softmax with cross-entropy for linear classifier
batch_size = 2
num_classes = 2
num_features = 3
np.random.seed(42)

In [18]:
W = np.random.randint(-1, 3, size=(num_features, num_classes)).astype(np.float)
print(W)

[[ 1.  2.]
 [-1.  1.]
 [ 1.  2.]]


In [19]:
X = np.random.randint(-1, 3, size=(batch_size, num_features)).astype(np.float)
print(X)

[[-1. -1.  1.]
 [ 0.  1.  1.]]


In [20]:
target_index = np.ones(batch_size, dtype=np.int)
print(target_index)

[1 1]


In [21]:
W = np.random.randint(-1, 3, size=(num_features, num_classes)).astype(np.float)
X = np.random.randint(-1, 3, size=(1, num_features)).astype(np.float)
target_index = np.ones(1, dtype=np.int)
predictions = np.dot(X, W)
print("X=", X, "\n out=", predictions, "\n target=",target_index)
pr_max=np.max(predictions,axis=1)
for i in range(predictions.shape[0]):
    predictions[i]=predictions[i]-pr_max[i]     
predictions=np.exp(predictions)
pr_sum=np.sum(predictions,axis=1)
for i in range(predictions.shape[0]):
    predictions[i]=predictions[i]/pr_sum[i]
print("softmax=", predictions)
loss= np.sum(- np.log(predictions[range(target_index.shape[0]),target_index.T])) / target_index.shape[0]
print("loss=", loss)
grad=predictions.copy()
grad[range(target_index.shape[0]),target_index.T]-=1
grad=grad/target_index.shape[0]
print("grad=", grad)
grad_w=np.dot(X.T,grad)
print("grad_w=", grad_w)

X= [[2. 1. 0.]] 
 out= [[4. 1.]] 
 target= [1]
softmax= [[0.95257413 0.04742587]]
loss= 3.048587351573742
grad= [[ 0.95257413 -0.95257413]]
grad_w= [[ 1.90514825 -1.90514825]
 [ 0.95257413 -0.95257413]
 [ 0.         -0.        ]]


In [22]:
W = np.random.randint(-1, 3, size=(num_features, num_classes)).astype(np.float)
X = np.random.randint(-1, 3, size=(batch_size, num_features)).astype(np.float)
target_index = np.ones(batch_size, dtype=np.int)
predictions = np.dot(X, W)
print("X=", X, "\n out=", predictions, "\n target=",target_index)
pr_max=np.max(predictions,axis=1)
for i in range(predictions.shape[0]):
    predictions[i]=predictions[i]-pr_max[i]     
predictions=np.exp(predictions)
pr_sum=np.sum(predictions,axis=1)
for i in range(predictions.shape[0]):
    predictions[i]=predictions[i]/pr_sum[i]
print("softmax=", predictions)
loss= np.sum(- np.log(predictions[range(target_index.shape[0]),target_index.T])) / target_index.shape[0]
print("loss=", loss)
grad=predictions.copy()
grad[range(target_index.shape[0]),target_index.T]-=1
grad=grad/target_index.shape[0]
print("grad=", grad)
grad_w=np.dot(X.T,grad)
print("grad_w=", grad_w)

X= [[ 0.  2.  2.]
 [-1. -1.  2.]] 
 out= [[ 4.  4.]
 [-1. -2.]] 
 target= [1 1]
softmax= [[0.5        0.5       ]
 [0.73105858 0.26894142]]
loss= 1.003204434039084
grad= [[ 0.25       -0.25      ]
 [ 0.36552929 -0.36552929]]
grad_w= [[-0.36552929  0.36552929]
 [ 0.13447071 -0.13447071]
 [ 1.23105858 -1.23105858]]


In [23]:
loss, dW = linear_classifer.linear_softmax(X, W, target_index)

### И теперь регуляризация

Мы будем использовать L2 regularization для весов как часть общей функции ошибки.

Напомним, L2 regularization определяется как

$$l2RegLoss = RegularizationStrength \cdot \sum_{i,j} w_{i, j}^{2}$$
т.е. сумма квадратов всех элементов, умноженная на скорость регуляризации. (В лекции было, что еще и корень от этого)

Реализуйте функцию для его вычисления и вычисления соотвествующих градиентов.

In [24]:
A= np.array([[1, 2], [3, 4]])
print(np.sum(A**2))

30


In [25]:
delta=1e-5
print(W)

[[-1.  0.]
 [ 2.  2.]
 [ 0.  0.]]


In [26]:
W1=W.copy()
W2=W.copy()
W1[0,0]+=delta
W2[0,0]-=delta
print(0.01*np.sum(((W1)**2)))
print(0.01*np.sum(((W2)**2)))
print(((0.01*np.sum(((W1)**2)))-(0.01*np.sum(((W2)**2))))/(2*delta))

0.089999800001
0.09000020000100001
-0.020000000000575113


In [27]:
2*0.01*W

array([[-0.02,  0.  ],
       [ 0.04,  0.04],
       [ 0.  ,  0.  ]])

In [28]:
# TODO Implement l2_regularization function that implements loss for L2 regularization
linear_classifer.l2_regularization(W, 0.01)

check_gradient(lambda w: linear_classifer.l2_regularization(w, 0.01), W)

Gradient check passed!


True

In [29]:
classifier = linear_classifer.LinearSoftmaxClassifier()

In [30]:
print(train_X, train_y)

[[-0.08975373 -0.03568431  0.08994824 ... -0.03998549  0.09710392
   1.        ]
 [ 0.01612863  0.01921765  0.05465412 ... -0.13410314 -0.10681765
   1.        ]
 [-0.25445961 -0.32980196 -0.36887529 ... -0.20469137 -0.23230784
   1.        ]
 ...
 [ 0.38083451  0.36823725  0.36053647 ...  0.23060275  0.21867255
   1.        ]
 [-0.2426949  -0.22391961 -0.12965961 ... -0.14194627 -0.13034706
   1.        ]
 [ 0.01220706  0.02706078  0.18798745 ...  0.06981843  0.10886863
   1.        ]] [9 9 2 ... 4 2 7]


In [31]:
num_train = train_X.shape[0]
num_features = train_X.shape[1]
print(train_X.shape, "количество тренировочных примеров и количество признаков")

(9000, 3073) количество тренировочных примеров и количество признаков


In [32]:
num_classes = np.max(train_y)+1
print(num_classes)

10


In [33]:
W = 0.001 * np.random.randn(num_features, num_classes)
print(W)

[[ 2.22218266e-05 -4.27792914e-04 -5.31817410e-04 ... -3.46521842e-05
   1.13433927e-03 -1.04745548e-04]
 [-5.25122851e-04  1.91277127e-03 -2.02671962e-03 ...  3.73118915e-04
  -3.86472951e-04 -1.15877024e-03]
 [ 5.66112827e-04 -7.04453450e-04 -1.37793930e-03 ...  1.20089277e-03
   6.98398941e-04 -1.71628835e-04]
 ...
 [-9.86647835e-05 -6.97203426e-04  1.78094324e-03 ... -1.52311984e-04
  -2.73694441e-03 -7.18960098e-04]
 [ 7.33053535e-04  1.12142976e-04 -9.31122362e-04 ... -7.23149546e-04
   1.80309484e-03  5.79680643e-04]
 [-8.05339206e-04  1.93344482e-05 -1.91584391e-04 ... -8.80196659e-04
   1.01721115e-03 -1.02844422e-03]]


In [34]:
print(W.shape)

(3073, 10)


In [35]:
shuffled_indices = np.arange(num_train)
print(shuffled_indices)

[   0    1    2 ... 8997 8998 8999]


In [36]:
np.random.shuffle(shuffled_indices)

In [37]:
shuffled_indices

array([5156, 4589,  450, ..., 1840, 2137, 6823])

In [38]:
batch_size=100
sections = np.arange(batch_size, num_train, batch_size)

In [39]:
sections

array([ 100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100,
       1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200,
       2300, 2400, 2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300,
       3400, 3500, 3600, 3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400,
       4500, 4600, 4700, 4800, 4900, 5000, 5100, 5200, 5300, 5400, 5500,
       5600, 5700, 5800, 5900, 6000, 6100, 6200, 6300, 6400, 6500, 6600,
       6700, 6800, 6900, 7000, 7100, 7200, 7300, 7400, 7500, 7600, 7700,
       7800, 7900, 8000, 8100, 8200, 8300, 8400, 8500, 8600, 8700, 8800,
       8900])

In [40]:
batches_indices = np.array_split(shuffled_indices, sections)

In [41]:
train_X[batches_indices[0]]

array([[-0.11328314, -0.20431176, -0.21985569, ...,  0.0109949 ,
         0.01867255,  1.        ],
       [ 0.02397176, -0.02784118, -0.08260078, ...,  0.02668118,
        -0.00877843,  1.        ],
       [ 0.08671686,  0.10549216,  0.0938698 , ...,  0.16785765,
         0.06573137,  1.        ],
       ...,
       [-0.06622431, -0.06313529, -0.12181647, ..., -0.00861294,
        -0.03622941,  1.        ],
       [ 0.12985412,  0.16823725,  0.15269333, ..., -0.07135804,
        -0.02838627,  1.        ],
       [ 0.47887373,  0.50941373,  0.52132078, ...,  0.52472039,
         0.50886863,  1.        ]])

In [42]:
train_X=np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],[17,18,19,20],[21,22,23,24],[25,26,27,28],
                  [29,1,5,6],[7,18,19,15]])
train_y=np.array([0,0,1,1,1,2,2,0,1])
batch_size=3
num_train = train_X.shape[0]
num_features = train_X.shape[1]
num_classes = np.max(train_y)+1

In [43]:
W = 0.001 * np.random.randn(num_features, num_classes)
print(W)

[[ 0.00066498  0.00150533  0.00031802]
 [-0.00120618  0.00064049  0.00081162]
 [-0.00070044  0.00106882  0.00045119]
 [-0.00199274 -0.00058798 -0.0003363 ]]


In [44]:
loss_history = []
epochs=1
shuffled_indices = np.arange(num_train)
np.random.shuffle(shuffled_indices)
sections = np.arange(batch_size, num_train, batch_size)
batches_indices = np.array_split(shuffled_indices, sections)


In [45]:
print(batches_indices)

[array([2, 4, 7]), array([1, 3, 5]), array([8, 6, 0])]


In [46]:
train_x1=train_X[batches_indices[0]]

In [47]:
print(train_x1)

[[ 9 10 11 12]
 [17 18 19 20]
 [29  1  5  6]]


In [48]:
train_y1=train_y[batches_indices[0]]
print(train_y1)

[1 1 0]


In [49]:
loss,grad=linear_classifer.linear_softmax(train_x1, W,train_y1)

In [50]:
learning_rate=1e-7

In [51]:
l2_loss,l2_grad=linear_classifer.l2_regularization(W,1e-5)

In [52]:
loss_history.append(loss+l2_loss)
W-=learning_rate*(grad+l2_grad)

In [53]:
W

array([[ 0.00066536,  0.00150557,  0.00031741],
       [-0.00120645,  0.00064109,  0.00081129],
       [-0.00070064,  0.00106941,  0.0004508 ],
       [-0.00199295, -0.00058735, -0.00033673]])

In [54]:
train_x1=train_X[batches_indices[1]]
train_y1=train_y[batches_indices[1]]
loss,grad=linear_classifer.linear_softmax(train_x1, W,train_y1)
l2_loss,l2_grad=linear_classifer.l2_regularization(W,1e-5)
loss_history.append(loss+l2_loss)
W-=learning_rate*(grad+l2_grad)
print(loss_history, "\n",W)

[1.0817116530390467, 1.0874791740342056] 
 [[ 0.00066512  0.00150555  0.00031767]
 [-0.00120669  0.00064107  0.00081155]
 [-0.00070088  0.00106939  0.00045105]
 [-0.00199318 -0.00058737 -0.00033647]]


In [55]:
train_x1=train_X[batches_indices[2]]
train_y1=train_y[batches_indices[2]]
loss,grad=linear_classifer.linear_softmax(train_x1, W,train_y1)
l2_loss,l2_grad=linear_classifer.l2_regularization(W,1e-5)
loss_history.append(loss+l2_loss)
W-=learning_rate*(grad+l2_grad)
print(loss_history, "\n",W)

[1.0817116530390467, 1.0874791740342056, 1.0813972454916663] 
 [[ 0.00066481  0.0015054   0.00031813]
 [-0.00120709  0.00064114  0.00081189]
 [-0.00070128  0.00106946  0.0004514 ]
 [-0.00199353 -0.00058742 -0.00033607]]


In [56]:
len(batches_indices)

3

In [57]:
train_X=np.array([[1,2,3,4,5],[5,6,7,8,3],[9,18,10,11,12],[13,20,14,15,16],[17,13,18,19,20],[21,29,22,23,24],[25,26,27,28,29],
                  [29,1,5,6,10],[7,18,19,15,11],[7,28,19,15,13]])
train_y=np.array([0,0,1,1,1,2,2,0,1,1])
batch_size=5
learning_rate=1e-2
num_train = train_X.shape[0]
num_features = train_X.shape[1]
num_classes = np.max(train_y)+1
W = 0.001 * np.random.randn(num_features, num_classes)
print(W, "\n", linear_classifer.softmax(np.dot(train_X, W)))
loss_history = []
epochs=100
for epoch in range(epochs):
    shuffled_indices = np.arange(num_train)
    np.random.shuffle(shuffled_indices)
    sections = np.arange(batch_size, num_train, batch_size)
    batches_indices = np.array_split(shuffled_indices, sections)
    for ind in range(len(batches_indices)):
        batch_train_x=train_X[batches_indices[ind]]
        batch_train_y=train_y[batches_indices[ind]]
        loss,grad=linear_classifer.linear_softmax(batch_train_x, W,batch_train_y)
        l2_loss,l2_grad=linear_classifer.l2_regularization(W,1e-5)
        loss_history.append(loss+l2_loss)
        W-=learning_rate*(grad+l2_grad)
        #print(loss_history, "\n",W)
    if epoch%10 ==0:
        print("Epoch %i, loss: %f" % (epoch, loss))
print(W,"\n", linear_classifer.softmax(np.dot(train_X, W)))

[[ 0.0006902   0.00137243 -0.00038609]
 [ 0.00017512  0.00031943  0.00084614]
 [-0.00153243  0.00150351  0.00023486]
 [ 0.00130612  0.00105529 -0.00213776]
 [ 0.00033364  0.00049145 -0.00021704]] 
 [[0.33344317 0.33674866 0.32980817]
 [0.33263893 0.34071726 0.32664381]
 [0.33169238 0.34461903 0.32368859]
 [0.331257   0.34929444 0.31944856]
 [0.33156428 0.35436692 0.3140688 ]
 [0.32983928 0.35849911 0.31166161]
 [0.32958954 0.36437345 0.30603701]
 [0.33455324 0.3465329  0.31891386]
 [0.32833647 0.34969744 0.32196609]
 [0.32754169 0.34946502 0.32299329]]
Epoch 0, loss: 2.877688
Epoch 10, loss: 7.886944
Epoch 20, loss: 1.603049
Epoch 30, loss: 3.335328
Epoch 40, loss: 3.240603
Epoch 50, loss: 1.203495
Epoch 60, loss: 4.440196
Epoch 70, loss: 3.403137
Epoch 80, loss: 1.265123
Epoch 90, loss: 1.921924
[[ 0.32127419 -0.52240198  0.20280425]
 [-0.31564373  0.31012976  0.00685459]
 [-0.04814332  0.1506856  -0.10233635]
 [ 0.22490969 -0.22359221 -0.00109384]
 [-0.25011428  0.0046511   0.2460712

In [58]:
print(W)

[[ 0.32127419 -0.52240198  0.20280425]
 [-0.31564373  0.31012976  0.00685459]
 [-0.04814332  0.1506856  -0.10233635]
 [ 0.22490969 -0.22359221 -0.00109384]
 [-0.25011428  0.0046511   0.24607121]]


In [59]:
linear_classifer.softmax(np.dot(train_X, W))

array([[1.04301573e-01, 1.69264620e-01, 7.26433806e-01],
       [3.27444402e-01, 4.91885905e-02, 6.23367008e-01],
       [4.59279205e-04, 2.01800216e-02, 9.79360699e-01],
       [1.66397787e-04, 8.92818355e-04, 9.98940784e-01],
       [1.07682628e-03, 2.52698216e-06, 9.98920647e-01],
       [4.19038414e-06, 7.66056327e-06, 9.99988149e-01],
       [6.01483119e-06, 5.90694280e-08, 9.99993926e-01],
       [4.44791350e-01, 4.61004477e-11, 5.55208650e-01],
       [1.68531044e-03, 3.08826440e-01, 6.89488250e-01],
       [5.34789302e-06, 8.51536751e-01, 1.48457901e-01]])

In [60]:
train_X=np.array([[1,0,0],[2,-1,0],[1,1,-2],[3,0,-1],
                  [4,3,-5],[2,-2,0],[5,-3,-2],[1,3,-2],
                  [3,-1,-1],[3,3,-5],[2,-2,2],[6,-3,-2]])
train_y=np.array([1,1,0,2,2,0,0,2,1,1,2,1])

In [61]:
num_train = train_X.shape[0]
num_features = train_X.shape[1]
num_classes = np.max(train_y)+1
print(num_train,num_features,num_classes)

12 3 3


In [62]:
W = 0.001 * np.random.randn(num_features, num_classes)
print(W)

[[ 9.36723628e-06 -9.76123020e-04  9.05553334e-04]
 [-8.05571259e-04  1.66621841e-03 -1.34615682e-03]
 [-3.04817577e-04 -2.48898894e-04 -1.11799789e-03]]


In [63]:
linear_classifer.softmax(np.dot(train_X, W))

array([[0.33334316, 0.33301481, 0.33364203],
       [0.33356652, 0.33208785, 0.33434563],
       [0.33296049, 0.33341845, 0.33362106],
       [0.33327778, 0.33227533, 0.33444688],
       [0.33230915, 0.33337218, 0.33431866],
       [0.33377982, 0.33147984, 0.33474033],
       [0.33384728, 0.32971512, 0.3364376 ],
       [0.33253091, 0.3346385 , 0.33283059],
       [0.33349104, 0.33166712, 0.33484184],
       [0.33229944, 0.33369113, 0.33400942],
       [0.3339491 , 0.33168505, 0.33436585],
       [0.333855  , 0.32939796, 0.33674704]])

In [64]:
epochs=1
batch_size=4
for epoch in range(epochs):
    shuffled_indices = np.arange(num_train)
    np.random.shuffle(shuffled_indices)
    sections = np.arange(batch_size, num_train, batch_size)
    batches_indices = np.array_split(shuffled_indices, sections)
    for ind in range(len(batches_indices)):
        batch_train_x=train_X[batches_indices[ind]]
        batch_train_y=train_y[batches_indices[ind]]
    

In [65]:
W_ist=np.array([[1,0,0],[1,0,0],[1,1,1]])
linear_classifer.softmax(np.dot(train_X, W_ist))

array([[5.76116885e-01, 2.11941558e-01, 2.11941558e-01],
       [5.76116885e-01, 2.11941558e-01, 2.11941558e-01],
       [7.86986042e-01, 1.06506979e-01, 1.06506979e-01],
       [9.09442999e-01, 4.52785007e-02, 4.52785007e-02],
       [9.98179556e-01, 9.10221936e-04, 9.10221936e-04],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [7.86986042e-01, 1.06506979e-01, 1.06506979e-01],
       [9.64663156e-01, 1.76684220e-02, 1.76684220e-02],
       [7.86986042e-01, 1.06506979e-01, 1.06506979e-01],
       [9.95066951e-01, 2.46652437e-03, 2.46652437e-03],
       [3.33333333e-01, 3.33333333e-01, 3.33333333e-01],
       [9.09442999e-01, 4.52785007e-02, 4.52785007e-02]])